In [177]:
import pdfplumber
import pandas as pd
import re
from datetime import datetime
import unicodedata

In [178]:
def committee_parser(file, congress_number):
    # Initialize an empty list to store data
    data = []
    partial_name = None  # Temporary storage for lines ending with "of" or comma
    ongoing_committee = None  # Temporary storage for incomplete committee names
    representative = None  # Store the current representative

    # Open and read the PDF
    with pdfplumber.open(file) as pdf:
        for page in pdf.pages:
            text = page.extract_text()

            # Split text by lines and iterate through
            lines = text.split('\n')

            for line in lines:
                # Debug: Print the current line being processed
                print(f"Processing line: {line.strip()}")

                # Identify and skip unwanted lines by pattern
                if any(skip in line for skip in [
                    'ASSIGNMENTS OF REPRESENTATIVES', 'COMMISSIONER, AND DELEGATES',
                    '[Democrats', 'and Delegates', 'Representative Committees', 'VerDate',
                    'htiw', 'Congressional Directory', '3G4F', 'BOJ_', 'House Committee Assignments'
                ]) or re.match(r'^\d+$', line.strip()):
                    continue

                # Check if the line is part of a multi-line representative name
                if partial_name:
                    # Combine with the previous line
                    line = f"{partial_name} {line.strip()}"
                    partial_name = None  # Reset the partial name

                # If the line ends with "of" or a comma, it is part of a multi-line representative name
                if line.strip().endswith("of") or line.strip().endswith(","):
                    partial_name = line.strip()
                    continue

                # Check if the line contains both a representative and their first committee
                if line.strip() and ('...' in line or '..' in line):
                    # Split the line into representative and committee parts
                    parts = re.split(r'\.{2,}', line.strip())
                    representative = parts[0].strip()
                    print(f"New representative: {representative}")
                    
                    # If there is a committee on the same line, capture it
                    if len(parts) > 1 and parts[1].strip():
                        first_committee = parts[1].strip()
                        
                        # Check for "Chair" or "Vice Chair" (case-insensitive)
                        rank = None
                        if ", chair" in first_committee.lower():
                            first_committee = first_committee.lower().replace(", chair", "").strip()
                            rank = "Chair"
                        elif ", vice chair" in first_committee.lower():
                            first_committee = first_committee.lower().replace(", vice chair", "").strip()
                            rank = "Vice Chair"
                        elif ", vicechair" in first_committee.lower():
                            first_committee = first_committee.lower().replace(", vicechair", "").strip()
                            rank = "Vice Chair"
                        elif ", co-chair" in first_committee.lower():
                            first_committee = first_committee.lower().replace(", co-chair", "").strip()
                            rank = "Co-Chair"
                        elif ",chair" in first_committee.lower():
                            first_committee = first_committee.lower().replace(",chair", "").strip()
                            rank = "Chair"
                        elif ",vice chair" in first_committee.lower():
                            first_committee = first_committee.lower().replace(",vice chair", "").strip()
                            rank = "Vice Chair"
                        elif ",vicechair" in first_committee.lower():
                            first_committee = first_committee.lower().replace(",vicechair", "").strip()
                            rank = "ViceChair"
                        elif ",co-chair" in first_committee.lower():
                            first_committee = first_committee.lower().replace(", co-chair", "").strip()
                            rank = "Co-Chair"

                        print(f"Adding first committee for {representative}: {first_committee} with rank: {rank}")
                        data.append([representative, first_committee, rank])
                    continue  # Move to the next line

                # Handle subsequent committee lines for the same representative
                elif representative and line.strip():
                    if ongoing_committee:
                        ongoing_committee += f" {line.strip()}"
                    else:
                        ongoing_committee = line.strip()

                    # Finalize committee if it ends with a period
                    if ongoing_committee.endswith("."):
                        # Check for "Chair" or "Vice Chair" (case-insensitive)
                        rank = None
                        if ", chair" in ongoing_committee.lower():
                            ongoing_committee = ongoing_committee.lower().replace(", chair", "").strip()
                            rank = "Chair"
                        elif ", vice chair" in ongoing_committee.lower():
                            ongoing_committee = ongoing_committee.lower().replace(", vice chair", "").strip()
                            rank = "Vice Chair"
                        elif ", vicechair" in ongoing_committee.lower():
                            ongoing_committee = ongoing_committee.lower().replace(", vicechair", "").strip()
                            rank = "Vice Chair"
                        elif ", co-chair" in ongoing_committee.lower():
                            ongoing_committee = ongoing_committee.lower().replace(", co-chair", "").strip()
                            rank = "Co-Chair"
                        elif ",chair" in ongoing_committee.lower():
                            ongoing_committee = ongoing_committee.lower().replace(",chair", "").strip()
                            rank = "Chair"
                        elif ",vice chair" in ongoing_committee.lower():
                            ongoing_committee = ongoing_committee.lower().replace(",vice chair", "").strip()
                            rank = "Vice Chair"
                        elif ",vicechair" in ongoing_committee.lower():
                            ongoing_committee = ongoing_committee.lower().replace(",vicechair", "").strip()
                            rank = "Vice Chair"
                        elif ",co-chair" in ongoing_committee.lower():
                            ongoing_committee = ongoing_committee.lower().replace(",co-chair", "").strip()
                            rank = "Co-Chair"

                        print(f"Adding committee for {representative}: {ongoing_committee.strip()} with rank: {rank}")
                        data.append([representative, ongoing_committee.strip(), rank])
                        ongoing_committee = None

                # Finalize the last committee for the representative on empty line
                if not line.strip():
                    if representative and ongoing_committee:
                        # Check for "Chair" or "Vice Chair" (case-insensitive)
                        rank = None
                        if ", chair" in ongoing_committee.lower():
                            ongoing_committee = ongoing_committee.lower().replace(", chair", "").strip()
                            rank = "Chair"
                        elif ", vice chair" in ongoing_committee.lower():
                            ongoing_committee = ongoing_committee.lower().replace(", vice chair", "").strip()
                            rank = "Vice Chair"
                        elif ", vicechair" in ongoing_committee.lower():
                            ongoing_committee = ongoing_committee.lower().replace(", vicechair", "").strip()
                            rank = "Vice Chair"
                        elif ", co-chair" in ongoing_committee.lower():
                            ongoing_committee = ongoing_committee.lower().replace(", co-chair", "").strip()
                            rank = "Co-Chair"
                        elif ",chair" in ongoing_committee.lower():
                            ongoing_committee = ongoing_committee.lower().replace(",chair", "").strip()
                            rank = "Chair"
                        elif ",vice chair" in ongoing_committee.lower():
                            ongoing_committee = ongoing_committee.lower().replace(",vice chair", "").strip()
                            rank = "Vice Chair"
                        elif ",vicechair" in ongoing_committee.lower():
                            ongoing_committee = ongoing_committee.lower().replace(",vicechair", "").strip()
                            rank = "Vice Chair"
                        elif ",co-chair" in ongoing_committee.lower():
                            ongoing_committee = ongoing_committee.lower().replace(",co-chair", "").strip()
                            rank = "Co-Chair"

                        print(f"Finalizing last committee for {representative}: {ongoing_committee.strip()} with rank: {rank}")
                        data.append([representative, ongoing_committee.strip(), rank])
                        ongoing_committee = None
                    representative = None  # Reset representative for the next block

    # Convert data into a DataFrame
    df = pd.DataFrame(data, columns=["Representative", "Committee", "Rank"])
    df['Congress'] = congress_number
    return df


In [179]:
df_117 = committee_parser("../data/committee_assignment_pdfs/COMMITTEEASSIGNMENTS-117.pdf",117)
df_116 = committee_parser("../data/committee_assignment_pdfs/COMMITTEEASSIGNMENTS-116.pdf",116)
df_115 = committee_parser("../data/committee_assignment_pdfs/COMMITTEEASSIGNMENTS-115.pdf",115)
df_115_2 = committee_parser("../data/committee_assignment_pdfs/COMMITTEEASSIGNMENTS-115_2.pdf",115)
df_114 = committee_parser("../data/committee_assignment_pdfs/COMMITTEEASSIGNMENTS-114.pdf",114)
df_113 = committee_parser("../data/committee_assignment_pdfs/COMMITTEEASSIGNMENTS-113.pdf",113)
df_112 = committee_parser("../data/committee_assignment_pdfs/COMMITTEEASSIGNMENTS-112.pdf",112)

Processing line: ASSIGNMENTS OF REPRESENTATIVES, RESIDENT
Processing line: COMMISSIONER, AND DELEGATES TO COMMITTEES
Processing line: [Democrats in roman (220); Republicans in italic (212); Vacancies (3); Resident Commissioner
Processing line: and Delegates in boldface (6); total, 441.]
Processing line: Representative Committees (Standing, Joint, and Select)
Processing line: Adams ........................................... Agriculture.
New representative: Adams
Adding first committee for Adams: Agriculture. with rank: None
Processing line: Education and Labor.
Adding committee for Adams: Education and Labor. with rank: None
Processing line: Financial Services.
Adding committee for Adams: Financial Services. with rank: None
Processing line: Aderholt ........................................ Appropriations.
New representative: Aderholt
Adding first committee for Aderholt: Appropriations. with rank: None
Processing line: Aguilar .......................................... Appropriations.
N

In [180]:
df_committee = pd.concat([df_117, df_116, df_115, df_115_2, df_114, df_113, df_112], axis=0, ignore_index=True)
df_committee

,Representative,Committee,Rank,Congress
0,Adams,Agriculture.,None,117
1,Adams,Education and Labor.,None,117
2,Adams,Financial Services.,None,117
3,Aderholt,Appropriations.,None,117
4,Aguilar,Appropriations.,None,117
...,...,...,...,...
5884,"Young, C. W. Bill, of Florida",Appropriations.,None,112
5885,"Young, Don, of Alaska",Natural Resources.,None,112
5886,"Young, Don, of Alaska",Transportation and Infrastructure.,None,112
5887,"Young, Todd C., of Indiana",Armed Services.,None,112


In [181]:
rep_df = pd.read_excel('../data/updated_legislator_table.xlsx')
rep_df

,chamber,startYear,endYear,name,bioguideId,partyName,state,district,startCongress,endCongress,lastName,firstName,middleName,aka,suffix
0,House of Representatives,2024,2024,"Wied, Tony",W000829,Republican,Wisconsin,8.0,118,118,Wied,Tony,NaN,NaN,NaN
1,House of Representatives,2024,2024,"Lee Carter, Erica",L000605,Democratic,Texas,18.0,118,118,Lee Carter,Erica,NaN,NaN,NaN
2,House of Representatives,1975,1985,"Patterson, Jerry M.",P000121,Democratic,California,38.0,94,99,Patterson,Jerry,M.,NaN,NaN
3,House of Representatives,1977,1993,"Ireland, Andrew P.",I000029,Republican,Florida,10.0,95,103,Ireland,Andrew,P.,NaN,NaN
4,House of Representatives,1991,2009,"Hobson, David L.",H000666,Republican,Ohio,7.0,102,111,Hobson,David,L.,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2787,House of Representatives,1975,1991,"Florio, James J.",F000215,Democratic,New Jersey,1.0,94,102,Florio,James,J.,NaN,NaN
2788,House of Representatives,1945,1947,"Flood, Daniel J.",F000209,Democratic,Pennsylvania,11.0,79,80,Flood,Daniel,J.,NaN,NaN
2789,House of Representatives,1949,1953,"Flood, Daniel J.",F000209,Democratic,Pennsylvania,11.0,81,83,Flood,Daniel,J.,NaN,NaN
2790,House of Representatives,1955,1981,"Flood, Daniel J.",F000209,Democratic,Pennsylvania,11.0,84,97,Flood,Daniel,J.,NaN,NaN


In [182]:
def parse_representative_name(name):
    # Initialize default values
    last_name = first_name = middle_initial = state = None

    # Check if there is a comma, indicating a full name format
    if ',' in name:
        # Split on the first comma to get last name and the rest of the name
        last_name, rest = name.split(',', 1)
        last_name = last_name.strip()

        # Split "rest" further to extract first name, middle initial, and state
        # Format: "First Middle Initial., of State"
        parts = rest.strip().split()
        
        # First name should be the first part
        first_name = parts[0].strip(",")

        # Check if there’s a middle initial with a period
        if len(parts) >= 2 and (parts[1].endswith('.') or parts[1].endswith('.,')):
            middle_initial = parts[1][0]  # Get just the initial without the period
            
        # Check if 'of' is in parts, then capture the state
        if 'of' in parts:
            state_index = parts.index('of') + 1
            # Join the remaining parts as the state (handles multi-word states like "North Carolina")
            state = " ".join(parts[state_index:])

    else:
        # If no comma, assume only the last name is provided
        last_name = name.strip()

    return last_name, first_name, middle_initial, state

# Apply the function to split the Representative column
df_committee[['Last Name', 'First Name', 'Middle Initial', 'State']] = df_committee['Representative'].apply(
    lambda x: pd.Series(parse_representative_name(x))
)

# Display the DataFrame
df_committee.head(20)  # Display the first few rows to confirm structure


,Representative,Committee,Rank,Congress,Last Name,First Name,Middle Initial,State
0,Adams,Agriculture.,None,117,Adams,None,None,None
1,Adams,Education and Labor.,None,117,Adams,None,None,None
2,Adams,Financial Services.,None,117,Adams,None,None,None
3,Aderholt,Appropriations.,None,117,Aderholt,None,None,None
4,Aguilar,Appropriations.,None,117,Aguilar,None,None,None
5,Aguilar,House Administration.,None,117,Aguilar,None,None,None
6,Aguilar,Select Committee to Investigate the January 6t...,None,117,Aguilar,None,None,None
7,Allen,Agriculture.,None,117,Allen,None,None,None
8,Allen,Education and Labor.,None,117,Allen,None,None,None
9,Allred,Foreign Affairs.,None,117,Allred,None,None,None


In [183]:
# Letters to correct
replacements = {
    'a´': 'á',
    '´a': 'á',
    'e´': 'é',
    '´e': 'é',
    'i´': 'í',
    '´i': 'í',
    'ı´': 'í',  # Dotless i
    '´ı': 'í',
    'o´': 'ó',
    '´o': 'ó',
    'u´': 'ú',
    '´u': 'ú',
    'A´': 'Á',
    '´A': 'Á',
    'E´': 'É',
    '´E': 'É',
    'I´': 'Í',
    '´I': 'Í',
    '´O': 'Ó',
    'U´': 'Ú',
    '´U': 'Ú',
    'O’': "O'"
}


# Apply the normalization function to the DataFrame
df_committee["Last Name"] = df_committee["Last Name"].replace(replacements, regex=True)
df_committee["First Name"] = df_committee["First Name"].replace(replacements, regex=True)

In [184]:
# Manual Correction to committee df
def correct_name(df, first_name=None, middle_initial=None, last_name=None, update_values=None):
    if update_values is None:
        update_values = {}
    
    # Start with a mask of all True values
    mask = pd.Series([True] * len(df), index=df.index)
    
    # Apply filters based on provided name components using regex
    if first_name is not None:
        pattern = re.escape(first_name)
        mask &= df['First Name'].str.contains(pattern, regex=True, na=False)
    if middle_initial is not None:
        mask &= df['Middle Initial'] == middle_initial
    if last_name is not None:
        mask &= df['Last Name'] == last_name
    
    # Check if any rows match the criteria
    if mask.any():
        # Update the specified columns with new values
        for column, value in update_values.items():
            if column in df.columns:
                df.loc[mask, column] = value
            else:
                print(f"Column '{column}' does not exist in the DataFrame.")
    else:
        print("No matching records found.")


correct_name(
    df_committee,
    first_name='Cathy',
    last_name='Rodgers',
    update_values={'Last Name': 'McMorris Rodgers'}
)

correct_name(
    df_committee,
    last_name='AMASH',
    update_values={'Last Name': 'Amash'}
)

correct_name(
    df_committee,
    last_name='BluntRochester',
    update_values={'Last Name': 'Blunt Rochester'}
)

correct_name(
    df_committee,
    last_name='HerreraBeutler',
    update_values={'Last Name': 'Herrera Beutler'}
)

correct_name(
    df_committee,
    last_name='JacksonLee',
    update_values={'Last Name': 'Jackson Lee'}
)

correct_name(
    df_committee,
    last_name='Luján',
    update_values={'Last Name': 'Lujan'}
)

correct_name(
    df_committee,
    first_name='CathyMcMorris,of',
    last_name='Rodgers',
    update_values={'Last Name': 'McMorris Rodgers', 'First Name': 'Cathy'}
)

correct_name(
    df_committee,
    last_name='SanNicolas',
    update_values={'Last Name': 'San Nicolas'}
)

correct_name(
    df_committee,
    last_name='TorresSmall',
    update_values={'Last Name': 'Torres Small'}
)

correct_name(
    df_committee,
    last_name='VanDrew',
    update_values={'Last Name': 'Van Drew'}
)
correct_name(
    df_committee,
    last_name='WassermanSchultz',
    update_values={'Last Name': 'Wasserman Schultz'}
)

correct_name(
    df_committee,
    last_name='WatsonColeman',
    update_values={'Last Name': 'Watson Coleman'}
)

correct_name(
    df_committee,
    last_name='LujanGrisham',
    update_values={'Last Name': 'Lujan Grisham'}
)

correct_name(
    df_committee,
    last_name='McMorrisRodgers',
    update_values={'Last Name': 'McMorris Rodgers'}
)

correct_name(
    df_committee,
    last_name='VanHollen',
    update_values={'Last Name': 'Van Hollen'}
)

correct_name(
    df_committee,
    last_name='NegreteMcLeod',
    update_values={'Last Name': 'Negrete McLeod'}
)

correct_name(
    df_committee,
    last_name='Barragan',
    update_values={'Last Name': 'Barragán'}
)

correct_name(
    df_committee,
    last_name='Gonzalez-Colon',
    update_values={'Last Name': 'González-Colón'}
)

correct_name(
    df_committee,
    last_name='Gutiérrez',
    update_values={'Last Name': 'Gutierrez'}
)

correct_name(
    df_committee,
    last_name='Cardenas',
    update_values={'Last Name': 'Cárdenas'}
)

correct_name(
    df_committee,
    last_name='Velazquez',
    update_values={'Last Name': 'Velázquez'}
)

No matching records found.


In [185]:
def find_bioguide(rep_df, df_row,index):
    # Find values
    congress = df_row['Congress']
    last = df_row['Last Name']
    first = df_row['First Name']
    middle = df_row['Middle Initial']
    state = df_row['State']
    #print(last)
    possibles = rep_df.loc[(rep_df['startCongress']<=congress) & (rep_df['endCongress']>=congress)]
    possibles = possibles.loc[possibles['lastName']==last]
    if len(possibles) == 1:
        return possibles['bioguideId'].iloc[0]
    elif len(possibles) > 1:
        possibles = possibles.loc[possibles['firstName']==first]
        if len(possibles) == 1:
            return possibles['bioguideId'].iloc[0]
    else:
        print("Multiple for : "+last)
        print(index)
        print(len(possibles))
    return
# print(df_committee.loc[2530])
# test_df =find_bioguide(rep_df,df2.loc[704], 704)
# test_df

for index, row in df_committee.iterrows():
    df_committee.at[index, 'bioguideID'] = find_bioguide(rep_df,row, index)

In [186]:
df_committee = df_committee.drop_duplicates()
df_committee

,Representative,Committee,Rank,Congress,Last Name,First Name,Middle Initial,State,bioguideID
0,Adams,Agriculture.,None,117,Adams,None,None,None,A000370
1,Adams,Education and Labor.,None,117,Adams,None,None,None,A000370
2,Adams,Financial Services.,None,117,Adams,None,None,None,A000370
3,Aderholt,Appropriations.,None,117,Aderholt,None,None,None,A000055
4,Aguilar,Appropriations.,None,117,Aguilar,None,None,None,A000371
...,...,...,...,...,...,...,...,...,...
5884,"Young, C. W. Bill, of Florida",Appropriations.,None,112,Young,C.,W,Florida,Y000031
5885,"Young, Don, of Alaska",Natural Resources.,None,112,Young,Don,None,Alaska,Y000033
5886,"Young, Don, of Alaska",Transportation and Infrastructure.,None,112,Young,Don,None,Alaska,Y000033
5887,"Young, Todd C., of Indiana",Armed Services.,None,112,Young,Todd,C,Indiana,Y000064


In [187]:
df_test = pd.DataFrame(df_committee['Committee'].str.lower().unique())
df_test

,0
0,agriculture.
1,education and labor.
2,financial services.
3,appropriations.
4,house administration.
...,...
123,democraticcongressionalcampaigncommittee.
124,financial services
125,joint select committee on deficit reduction.
126,no committee assignments.


In [188]:
# List of strings to remove
string_list = [
    '[no committee assignments at press time.]',
    'greene, marjorie taylor, of [no committee assignments at press time.] georgia.',
    'johnson, dusty, of south dakota agriculture.',
    'johnson, eddie bernice, of texas science, space, and technology.',
    '[nocommitteeassignmentsatpresstime.]',
    'johnson,dusty,ofsouthdakota agriculture.',
    'johnson,eddiebernice,oftexas science,space,andtechnology.',
    'johnson,henryc.â€˜â€˜hankâ€™â€™,jr., judiciary.',
    'seniorchiefdeputymajoritywhip.',
    '´','assistant democratic leader.',
    'jenkins, evan h., of west appropriations.',
    'virginia.','price, david e., of north carolina appropriations.',
    'jenkins,evanh.,ofwest appropriations.',
    'price,davide.,ofnorthcarolina appropriations.',
    'ellmers,reneel.,ofnorth energyandcommerce.',
    'carolina.','bishop,timothyh.,ofnewyork educationandtheworkforce.',
    'no committee assignments.',
    'mccarthy, carolyn, of new york education and the workforce.',
    'ch',
    'johnson,henryc.‘‘hank’’,jr., judiciary',
    'johnson,henryc.‘‘hank’’,jr., judiciary.'
]

# Remove rows where 'column_name' equals any string in 'string_list'
df_committee = df_committee[~df_committee['Committee'].str.lower().isin([s.lower() for s in string_list])]

In [189]:
def correct_misspellings(text, correction_dict):
    """
    Corrects the text if it matches a misspelling in the correction dictionary.
    The function is case-insensitive and preserves the original casing.
    
    Parameters:
    - text (str): The text to check and possibly correct.
    - correction_dict (dict): A dictionary mapping misspellings to corrections.
    
    Returns:
    - str: The corrected text if it was a misspelling, otherwise the original text.
    """
    # Ensure the input is a string
    if not isinstance(text, str):
        return text
    
    # Convert the text to lowercase for case-insensitive comparison
    lower_text = text.lower()
    
    # Check if the entire text matches any misspelling
    if lower_text in correction_dict:
        correct_word = correction_dict[lower_text]
        # Preserve the original casing
        if text.isupper():
            return correct_word.upper()
        elif text[0].isupper():
            return correct_word.capitalize()
        else:
            return correct_word
    else:
        # Return the original text if no correction is needed
        return text

# Dictionary of misspellings and corrections
correction_dict = {
    'select committee on economic disparity and fairness in': 'select committee on economic disparity and fairness in growth',
    'growth.': 'select committee on economic disparity and fairness in growth',
    'armedservices.': 'armed services.',
    'educationandlabor.': 'education and labor.',
    'financialservices.': 'financial services.',
    'houseadministration.': 'house administration.',
    'foreignaffairs.': 'foreign affairs.',
    'transportationandinfrastructure.': 'transportation and infrastructure.',
    "veterans'affairs.": "veterans' affairs.",
    'oversightandreform.': 'oversight and reform.',
    'selectcommitteeontheclimatecrisis.': 'select committee on the climate crisis.',
    'waysandmeans.': 'ways and means.',
    'science,space,andtechnology.': 'science, space ,and technology.',
    'smallbusiness.': 'small business.',
    'energyandcommerce.': 'energy and commerce.',
    'homelandsecurity.': 'homeland security.',
    'naturalresources.': 'natural resources.',
    'selectcommitteeonthemodernizationofcongress.': 'select committee on the modernization of congress.',
    'permanentselectcommitteeonintelligence.': 'permanent select committee on intelligence.',
    'ofgeorgia. transportationandinfrastructure.': 'transportation and infrastructure.',
    'houseadministrationperson.': 'house administration.',
    'oversightandreformwoman.': 'oversight and reform.',
    'permanentselectcommitteeonintelligenceman.': 'permanent select committee on intelligence.',
    "education and the workforce veterans' affairs.": 'education and the workforce',
    '´ natural resources.': 'natural resources.',
    '´ budget.': 'budget.',
    'joint economic committee armed services.': 'joint economic committee',
    '´ ´ small business.': 'small business.',
    "veterans' affairs foreign affairs.": 'foreign affairs.',
    '´ oversight and government reform.': 'oversight and government reform.',
    'educationandtheworkforce.': 'education and the workforce.',
    'jointeconomiccommittee.': 'joint economic committee.',
    'oversightandgovernmentreform.': 'oversight and government reform.',
    "educationandtheworkforce veterans'affairs.": 'education and the workforce',
    'jointcommitteeontaxation.': 'joint committee on taxation.',
    'jointcommitteeonthelibrary.': 'joint committee on the library.',
    'jointcommitteeonprinting.': 'joint committee on printing.',
    'jointeconomiccommittee armedservices.': 'armed services.',
    'jointcommitteeonthetaxation.': 'joint committee on the taxation.',
    "veterans'affairs foreignaffairs": 'foreignaffairs',
    'select committee on the events surrounding the 2012 terroristattackinbenghazi.': 'select committee on the events surrounding the 2012 terrorist attack in benghazi.',
    'select committee on the events surrounding the 2012': 'select committee on the events surrounding the 2012 terrorist attack in benghazi.',
    'terroristattackinbenghazi.': 'select committee on the events surrounding the 2012 terrorist attack in benghazi.',
    'apropriations.': 'appropriations.',
    'ehtics.': 'ethics.',
    'democraticcongressionalcampaigncommittee.': 'democratic congressional campaign committee.',
    'joint select committee on deficit reduction.': 'joint select committee on deficit reduction.',
    'houseadministration,rankingmember.':'house administration',
    'science,space,andtechnology,rankingmember.':'science, space, and technology',
    'veterans’affairs,rankingmember.':'veterans’ affairs',
    'rules,rankingmember.':'rules',
    'educationandtheworkforce veterans’affairs.':'education and the workforce',
    'budget, vice chiar':'budget',
    'veterans’affairs foreignaffairs':'foreign affairs',
    'budget,vicechiar':'budget',
    'the speaker':'speaker',
    'thespeaker':'speaker'
}

# Apply the correction function to the DataFrame column
df_committee['Committee'] = df_committee['Committee'].apply(lambda x: correct_misspellings(x, correction_dict))

C:\Users\Jerrin\AppData\Local\Temp\ipykernel_44168\2038224379.py:96: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_committee['Committee'] = df_committee['Committee'].apply(lambda x: correct_misspellings(x, correction_dict))


In [190]:
df_test = pd.DataFrame(df_committee['Committee'].str.lower().unique())
df_test

,0
0,agriculture.
1,education and labor.
2,financial services.
3,appropriations.
4,house administration.
...,...
66,"budget,vicechiar."
67,select committee on the events surrounding the...
68,democratic congressional campaign committee.
69,financial services


In [191]:
df_committee['Committee'] = df_committee['Committee'].str.rstrip('.')
# Apply the correction function to the DataFrame column
df_committee['Committee'] = df_committee['Committee'].apply(lambda x: correct_misspellings(x, correction_dict))
df_committee['Committee'].str.lower().unique()

C:\Users\Jerrin\AppData\Local\Temp\ipykernel_44168\1239603817.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_committee['Committee'] = df_committee['Committee'].str.rstrip('.')
C:\Users\Jerrin\AppData\Local\Temp\ipykernel_44168\1239603817.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_committee['Committee'] = df_committee['Committee'].apply(lambda x: correct_misspellings(x, correction_dict))


array(['agriculture', 'education and labor', 'financial services',
       'appropriations', 'house administration',
       'select committee to investigate the january 6th attack on the united states capitol',
       'foreign affairs', 'transportation and infrastructure',
       'veterans’ affairs', 'energy and commerce', 'ethics',
       'select committee on the climate crisis', 'ways and means',
       'select committee on economic disparity and fairness in growth',
       'science, space, and technology', 'armed services',
       'homeland security', 'judiciary', 'natural resources',
       'oversight and reform', 'budget', 'small business', 'rules',
       'permanent select committee on intelligence', 'assistant speaker',
       'select committee on the modernization of congress',
       'majority whip', 'majority leader', 'republican leader', 'speaker',
       'republican whip', 'veterans’affairs',
       'science, space ,and technology', 'majoritywhip',
       'select committee o

In [192]:
len(df_committee['Committee'].str.lower().unique())

57

In [193]:
df_committee.to_excel('../data/committee_assignments.xlsx',index=False)